In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import random
import string
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import _linear
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import collections
import matplotlib.pyplot as plt
import codecs
import time
import os
import gc
from six.moves import cPickle as pickle

from plot_module import text_plot
from plot_module import structure_vocabulary_plots
from plot_module import ComparePlots

from model_module import maybe_download
from model_module import read_data
from model_module import check_not_one_byte
from model_module import id2char
from model_module import char2id
from model_module import BatchGenerator
from model_module import characters
from model_module import batches2string
from model_module import logprob
from model_module import sample_distribution
from model_module import MODEL

In [3]:
if not os.path.exists('enwik8_filtered'):
    if not os.path.exists('enwik8'):
        filename = maybe_download('enwik8.zip', 36445475)
    full_text = read_data(filename)
    new_text = u""
    new_text_list = list()
    for i in range(len(full_text)):
        if (i+1) % 10000000 == 0:
            print("%s characters are filtered" % i)
        if ord(full_text[i]) < 256:
            new_text_list.append(full_text[i])
    text = new_text.join(new_text_list)
    del new_text_list
    del new_text
    del full_text

    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)
    
    f = open('enwik8_filtered', 'w')
    f.write(text.encode('utf8'))
    f.close()
    
else:
    f = open('enwik8_filtered', 'r')
    text = f.read().decode('utf8')
    f.close() 
    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)    

number of not one byte characters:  0
min order index:  9
max order index:  255
total number of characters:  196


In [4]:
#different
offset = 30000
valid_size = 25000
valid_text = text[offset:offset+valid_size]
train_text = text[offset+valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99325000 c facade.&lt;ref&gt;[http://library.nothingness.org/articles/SI/
25000 n in the February 1934 riots, anarchists divided over a 'united 


In [5]:
#different
offset_1 = 0
offset_2 = 4100
valid_size_1 = 4000
valid_size_2 = 4000
valid_text_1 = text[offset_1:offset_1+valid_size_1]
valid_text_2 = text[offset_2:offset_2+valid_size_2]
train_text = text[offset_2+valid_size_2:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size_1, valid_text_1[:64])
print(valid_size_2, valid_text_2[:64])
print(valid_text_1)
print('\n\n\n')
print(valid_text_2)

99371900 , Zeno &quot;repudiated the omnipotence of the state, its interv
4000 <mediawiki xmlns="http://www.mediawiki.org/xml/export-0.3/" xmln
4000 reserve">{{Anarchism}}
'''Anarchism''' originated as a term of a
<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.3/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.3/ http://www.mediawiki.org/xml/export-0.3.xsd" version="0.3" xml:lang="en">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <base>http://en.wikipedia.org/wiki/Main_Page</base>
    <generator>MediaWiki 1.6alpha</generator>
    <case>first-letter</case>
      <namespaces>
      <namespace key="-2">Media</namespace>
      <namespace key="-1">Special</namespace>
      <namespace key="0" />
      <namespace key="1">Talk</namespace>
      <namespace key="2">User</namespace>
      <namespace key="3">User talk</namespace>
      <namespace key="4">Wikipedia</namespace>
      <namespace key="5">Wikipedia talk</n

In [6]:
vocabulary_size = number_of_characters
vocabulary = list()
characters_positions_in_vocabulary = list()

character_position_in_vocabulary = 0
for i in range(256):
    if present_characters_indices[i]:
        vocabulary.append(unichr(i))
        characters_positions_in_vocabulary.append(character_position_in_vocabulary)
        character_position_in_vocabulary += 1
    else:
        characters_positions_in_vocabulary.append(-1)


string_vocabulary = u""
for i in range(vocabulary_size):
    string_vocabulary += vocabulary[i]
print("Vocabulary: ", string_vocabulary)
print("char2id(u'a') = %s,  char2id(u'z') = %s,  char2id(u' ') = %s" % (char2id(u'a', characters_positions_in_vocabulary),
                                                                        char2id(u'z', characters_positions_in_vocabulary),
                                                                        char2id(u' ', characters_positions_in_vocabulary)))
print("id2char(78) = %s,  id2char(156) = %s,  id2char(140) = %s" % (id2char(78,
                                                                            vocabulary),
                                                                    id2char(156,
                                                                            vocabulary),
                                                                    id2char(140,
                                                                            vocabulary)))

Vocabulary:  	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ
char2id(u'a') = 67,  char2id(u'z') = 92,  char2id(u' ') = 2
id2char(78) = l,  id2char(156) = Ø,  id2char(140) = È


In [7]:
batch_size_test=64
num_unrollings_test=10

train_batches_test = BatchGenerator(train_text,
                                    batch_size_test,
                                    vocabulary_size,
                                    characters_positions_in_vocabulary,
                                    num_unrollings_test)
valid_batches_test = BatchGenerator(valid_text_1,
                                    1,
                                    vocabulary_size,
                                    characters_positions_in_vocabulary,
                                    1)

print(batches2string(train_batches_test.next(), vocabulary))
print(batches2string(train_batches_test.next(), vocabulary))
print(batches2string(valid_batches_test.next(), vocabulary))
print(batches2string(valid_batches_test.next(), vocabulary))

[u', Zeno &quo', u'act science', u'] systems. ', u"* ''Women E", u'nis. It ori', u'ilanthus al', u'ave diminis', u't;Advance A', u'orded in ce', u' late [[179', u'. A strong ', u'bodia]] led', u'f the Hill]', u']] | above=', u"t rather ''", u"d 'Babe Rut", u'[Chicago Wh', u'overed to b', u'when he spr', u' search].\n*', u'erminology=', u'ile.\n\n====1', u'ife and car', u'509</id>\n  ', u' featured i', u'other marin', u'ere has bee', u' be copied ', u' subdivisio', u'eased drawi', u'include:\n\n[', u'for the thi', u'age do not ', u'ion>\n      ', u'(book)|Drag', u': ([[1921]]', u'ndia in ord', u'how can we ', u'</timestamp', u's and also ', u'n ==\nThe Es', u'nredlist.or', u' fully or p', u'ma was revi', u'ometimes &l', u'when there ', u' and [[pena', u'arkets (see', u'uage with t', u'the [[Ameri', u'lable power', u'ername>\n   ', u'phen]]s in ', u'ws and the ', u' culture ha', u'uardians of', u'llaboration', u'ot;Lords of', u' Howard, 3r', u'ersonnel ca', u' drafts [[l', u'edarville, '

In [14]:

class ACT_LSTM_individual_2(MODEL):
    
    def LSTM_cell(self, inp, state):
        #print('inp.shape: ', inp.get_shape().as_list())
        #print('state[0].shape: ', state[0].get_shape().as_list())
        X = tf.concat(1, [inp,
                          state[0]])
        concat = tf.matmul(X, self.Matrix) + self.Bias
        [i, f, o, g] = tf.split(1, 4, concat)
        i = tf.sigmoid(i)
        f = tf.sigmoid(f)
        o = tf.sigmoid(o)
        g = tf.tanh(g)
        new_state = tf.tanh(i*g+f*state[1])
        output = o * new_state 
        return [output, new_state]
    
    def add_flag_to_input(self, X, idx):
        flags = list()
        one_flag = [0.]*self._depth
        one_flag[idx] = 1.
        flag = tf.pack([tf.constant(one_flag)] * X.get_shape().as_list()[0])
        return tf.concat(1, [X, flag])
    
    def compute_halting_units(self, intermediate_states):
        states_concat = tf.concat(0, [tf.concat(1, [state[0], state[1]]) for state in intermediate_states])
        halting_units = tf.sigmoid(tf.matmul(states_concat, self.Halting_Matrix) + self.Halting_Bias)
        return tf.split(0, len(intermediate_states), halting_units)
        
    
    def compute_probabilities(self, halting_units):
        b_size = halting_units[0].get_shape().as_list()[0]
        agregate = tf.zeros([b_size, self._num_nodes[0]])
        completed = tf.zeros([b_size, self._num_nodes[0]])
        ones = tf.ones([b_size, self._num_nodes[0]])
        minus_ones = - ones
        probabilities = list()
        prob_loss = 0
        for idx, halting_unit in enumerate(halting_units):
            if idx == self._depth - 1:
                completed_flags = tf.to_float(tf.greater(agregate, -0.5))
            else:
                completed_flags = tf.to_float(tf.greater(agregate + halting_unit, self._threshold))
            completed += completed_flags
            probabilities.append((ones - completed) * halting_unit + completed_flags * (ones - agregate))
            prob_loss += tf.reduce_sum(completed_flags * (ones - agregate))
            agregate = minus_ones * completed + (ones - completed) * (agregate + halting_unit)
        return probabilities, prob_loss
            
    def compute_number_of_iterations(self, probabilities):
        probs = tf.pack(probabilities, axis=2)
        looped = tf.greater(probs, 0)
        looped = tf.to_int32(looped)
        return tf.reduce_sum(looped, reduction_indices=[2])
               
    
    def iteration(self, inp, old_state):
        """print('is_list: ', isinstance(old_state, list))
        if isinstance(old_state, list):
            print('old_state[0].shape: ', old_state[0].get_shape().as_list())
        else:
            print('old_state.shape: ', old_state.get_shape().as_list())"""
        b_size = inp.get_shape().as_list()[0]
        intermediate_states = list()
        halting_units = list()
        agregate = tf.zeros([b_size, self._num_nodes[0]])
        ones = tf.ones([b_size, self._num_nodes[0]])
        for idx in range(self._depth):
            new_state = self.LSTM_cell(self.add_flag_to_input(inp, idx), old_state)
            mask = tf.to_float(tf.less(agregate, self._threshold))
            very_old_state = old_state
            old_state = list()
            old_state.append(mask * new_state[0] + (ones - mask) * very_old_state[0])
            old_state.append(mask * new_state[1] + (ones - mask) * very_old_state[1])
            halting_unit = tf.sigmoid(tf.matmul(tf.concat(1, old_state), self.Halting_Matrix) + self.Halting_Bias)
            agregate += halting_unit
            halting_units.append(halting_unit)
            intermediate_states.append(old_state)
        probabilities, prob_loss = self.compute_probabilities(halting_units)
        number_of_iterations = self.compute_number_of_iterations(probabilities)
        new_state = [0, 0]
        for inter_state, probability in zip(intermediate_states, probabilities):
            new_state[0] +=  inter_state[0] * probability
            new_state[1] +=  inter_state[1] * probability
        return new_state[0], new_state, prob_loss, number_of_iterations, probabilities, halting_units

    
    def __init__(self,
                 batch_size,
                 vocabulary,
                 characters_positions_in_vocabulary,
                 num_unrollings,
                 num_nodes,
                 depth,
                 indent,
                 time_penalty,
                 train_text,
                 valid_text,
                 mean=0.,
                 stddev='default',
                 shift=0.,
                 init_learning_rate=5.):
        self._results = list()
        self._batch_size = batch_size
        self._vocabulary = vocabulary
        self._vocabulary_size = len(vocabulary)
        self._characters_positions_in_vocabulary = characters_positions_in_vocabulary
        
        self._num_unrollings = num_unrollings

        self._num_layers = 1
        self._num_nodes = num_nodes
        self._depth = depth
        self._indent = indent
        self._time_penalty = time_penalty
        self._threshold = 1. - indent
        self._train_text = train_text
        self._valid_text = valid_text
        if isinstance(self._valid_text, dict):
            keys = self._valid_text.keys()
            self._valid_size = dict([zipped for zipped in zip(keys, [len(self._valid_text[key]) for key in keys])])
        else:
            self._valid_size = len(valid_text)
        
        
        self._mean = mean
        
        self._shift = shift
        
        self._stddev = list()
        if stddev == 'default':
            self._stddev = 1.
        else:
            self._stddev = stddev

        self._init_learning_rate = init_learning_rate
  
        self._indices = {"batch_size": 0,
                         "num_unrollings": 1,
                         "num_layers": 2,
                         "num_nodes": 3,
                         "depth": 4,
                         "indent": 5,
                         "time_penalty": 6,
                         "half_life": 7,
                         "decay": 8,
                         "num_steps": 9,
                         "averaging_number": 10,
                         "init_mean": 11,
                         "init_stddev": 12,
                         "init_shift": 13,
                         "init_learning_rate": 14,                         
                         "type": 15}
        self._graph = tf.Graph()
        
        self._last_num_steps = 0
        with self._graph.as_default(): 
            with self._graph.device('/gpu:0'): 
                self.Matrix = tf.Variable(tf.truncated_normal([self._vocabulary_size + self._num_nodes[0] + self._depth,
                                                                      4 * self._num_nodes[0]],
                                                                     mean=self._mean, stddev=self._stddev / (self._vocabulary_size + self._num_nodes[0] + self._depth)**0.5))
                self.Bias = tf.Variable(tf.zeros([4 * self._num_nodes[0]]))
                
                self.Halting_Matrix = tf.Variable(tf.truncated_normal([2 * self._num_nodes[0],
                                                                      self._num_nodes[0]],
                                                                     mean=self._mean, stddev=self._stddev / self._num_nodes[0]**0.5))
                
                self.Halting_Bias = tf.Variable(tf.zeros([self._num_nodes[0]]))
                # classifier 
                weights = tf.Variable(tf.truncated_normal([self._num_nodes[-1], self._vocabulary_size], stddev = self._stddev / self._num_nodes[-1]**0.5))
                bias = tf.Variable(tf.zeros([self._vocabulary_size]))
                
                """PLACEHOLDERS train data"""
                self._train_data = list()
                for _ in range(self._num_unrollings + 1):
                    self._train_data.append(
                        tf.placeholder(tf.float32, shape=[self._batch_size, self._vocabulary_size]))
                    
                    
                train_inputs = self._train_data[: self._num_unrollings]
                train_labels = self._train_data[1:]  # labels are inputs shifted by one time step.
                # Unrolled LSTM loop.
                
                """global step"""
                self._global_step = tf.Variable(0, trainable=False)

                saved_state = list()
                saved_state.append(tf.Variable(tf.zeros([self._batch_size, self._num_nodes[0]]), trainable=False))
                saved_state.append(tf.Variable(tf.zeros([self._batch_size, self._num_nodes[0]]), trainable=False))

                outputs = list()
                state = saved_state
                self.number_of_iterations = list()
                first_inputs = list()
                first_outputs = list()
                for i in train_inputs:
                    output, state, prob_loss, number_of_iterations, _, _ = self.iteration(i, state)
                    self.number_of_iterations.append(tf.unpack(number_of_iterations)[0])
                    outputs.append(output)
                    first_inputs.append(tf.split(0, self._batch_size, i)[0])
                    first_outputs.append(tf.split(0, self._batch_size, output)[0])
                self.first_inputs = tf.concat(0, first_inputs)
                self.first_outputs = tf.nn.softmax(tf.nn.xw_plus_b(tf.concat(0, first_outputs), weights, bias))
                self.number_of_iterations = tf.pack(self.number_of_iterations)    

                save_list = list()
                save_list.append(saved_state[0].assign(state[0]))
                save_list.append(saved_state[1].assign(state[1]))
                
                """skip operation"""
                self._skip_operation = tf.group(*save_list)

                with tf.control_dependencies(save_list):
                        # Classifier.
                    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), weights, bias)
                    """loss"""
                    self._loss = tf.reduce_mean(
                        tf.nn.softmax_cross_entropy_with_logits(
                        logits, tf.concat(0, train_labels))) + self._time_penalty * prob_loss / self._num_nodes

                # Optimizer.
                
                """PLACEHOLDERS half life and decay"""
                self._half_life = tf.placeholder(tf.int32)
                self._decay = tf.placeholder(tf.float32)
                """learning rate"""
                self._learning_rate = tf.train.exponential_decay(self._init_learning_rate,
                                                                 self._global_step,
                                                                 self._half_life,
                                                                 self._decay,
                                                                 staircase=True)
                
                optimizer = tf.train.GradientDescentOptimizer(self._learning_rate)
                
                halting_vars = [self.Halting_Matrix, self.Halting_Bias]
                all_vars = [self.Matrix, self.Bias, weights, bias, self.Halting_Matrix, self.Halting_Bias]
                
                halting_grads, halting_vars = zip(*optimizer.compute_gradients(prob_loss, var_list=halting_vars))
                all_grads, all_vars = zip(*optimizer.compute_gradients(self._loss, var_list=all_vars))
                
                halting_dict = dict(zip(halting_vars, halting_grads))
                all_dict = dict(zip(all_vars, all_grads))
                
                for halting_key in halting_dict.keys():
                    all_dict[halting_key] += halting_dict[halting_key]
                
                all_keys = all_dict.keys()
                for key in all_keys:
                    help_var, _ = tf.clip_by_global_norm([all_dict[key]], 1.25)
                    all_dict[key] = help_var[0]

                """optimizer"""
                self._optimizer = optimizer.apply_gradients(zip([all_dict[key] for key in all_keys], [key for key in all_keys]), global_step=self._global_step)
                """train prediction"""
                self._train_prediction = tf.nn.softmax(logits)


                # Sampling and validation eval: batch 1, no unrolling.
                # Sampling and validation eval: batch 1, no unrolling.
                saved_sample_state = list()
                saved_sample_state.append(tf.Variable(tf.zeros([1, self._num_nodes[0]]), trainable=False))
                saved_sample_state.append(tf.Variable(tf.zeros([1, self._num_nodes[0]]), trainable=False)) 
                """PLACEHOLDER sample input"""
                self._sample_input = tf.placeholder(tf.float32, shape=[1, self._vocabulary_size])

                reset_list = list()
                reset_list.append(saved_sample_state[0].assign(tf.zeros([1, self._num_nodes[0]])))
                reset_list.append(saved_sample_state[1].assign(tf.zeros([1, self._num_nodes[0]])))

                """reset sample state"""
                self._reset_sample_state = tf.group(*reset_list)

                sample_output, sample_state, _, self.num_iter, self.probs, hu = self.iteration(self._sample_input, saved_sample_state)

                self.halting_units = tf.concat(0, hu)
                sample_save_list = list()
                sample_save_list.append(saved_sample_state[0].assign(sample_state[0]))
                sample_save_list.append(saved_sample_state[1].assign(sample_state[1]))

                with tf.control_dependencies(sample_save_list):
                    """sample prediction"""
                    self._sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, weights, bias)) 
                
                
                """saver"""
                self.saver = tf.train.Saver(max_to_keep=None)
                            
                        
    
    def _generate_metadata(self, half_life, decay, num_averaging_iterations):
        metadata = list()
        metadata.append(self._batch_size)
        metadata.append(self._num_unrollings)   
        metadata.append(self._num_layers)
        metadata.append(self._num_nodes)
        metadata.append(self._depth)
        metadata.append(self._indent)
        metadata.append(self._time_penalty)
        metadata.append(half_life)
        metadata.append(decay)
        metadata.append(self._last_num_steps)
        metadata.append(num_averaging_iterations)
        metadata.append(self._mean)
        metadata.append(self._stddev)
        metadata.append(self._shift)
        metadata.append(self._init_learning_rate)
        metadata.append('ACT_LSTM_individual_2')
        return metadata

    def get_iterations(self, session, num_strings=10, length=75, start_positions=None):
        self._reset_sample_state.run()
        self._valid_batches = BatchGenerator(self._valid_text,
                                             1,
                                             self._vocabulary_size,
                                             self._characters_positions_in_vocabulary,
                                             1)
        if start_positions is None:
            start_positions = list()
            if self._valid_size / num_strings < length:
                num_strings = self._valid_size / length
            for i in range(num_strings):
                start_positions.append(i* (self._valid_size / num_strings) + self._valid_size / num_strings / 2)
            while self._valid_size - start_positions[-1] < length:
                del start_positions[-1]
        text_list = list()
        iterations_list = list()
        collect_iterations = False
        letters_parsed = -1
        for idx in range(self._valid_size):
            b = self._valid_batches.next()
            
            if idx in start_positions or collect_iterations: 
                if letters_parsed == -1:
                    letters_parsed = 0
                    text = u""
                    i_list = [list() for _ in range(self._num_nodes[0])]
                    collect_iterations = True
                text += characters(b[0], self._vocabulary)[0]
                for_distribution = self.num_iter.eval({self._sample_input: b[0]})
                for node_idx in range(for_distribution.shape[1]):
                    i_list[node_idx].append(for_distribution[0][node_idx])
                letters_parsed += 1
                if letters_parsed >= length:
                    collect_iterations = False
                    iterations_list.append(i_list)
                    text_list.append(text)
                    letters_parsed = -1
                    
            _ = self._sample_prediction.eval({self._sample_input: b[0]})
        return text_list, iterations_list  
    
    def get_average_iterations(self, session, num_strings=10, length=75, start_positions=None):
        self._reset_sample_state.run()
        self._valid_batches = BatchGenerator(self._valid_text,
                                             1,
                                             self._vocabulary_size,
                                             self._characters_positions_in_vocabulary,
                                             1)
        if start_positions is None:
            start_positions = list()
            if self._valid_size / num_strings < length:
                num_strings = self._valid_size / length
            for i in range(num_strings):
                start_positions.append(i* (self._valid_size / num_strings) + self._valid_size / num_strings / 2)
            while self._valid_size - start_positions[-1] < length:
                del start_positions[-1]
        text_list = list()
        iterations_list = list()
        collect_iterations = False
        letters_parsed = -1
        for idx in range(self._valid_size):
            b = self._valid_batches.next()
            
            if idx in start_positions or collect_iterations: 
                if letters_parsed == -1:
                    letters_parsed = 0
                    text = u""
                    i_list = list()
                    collect_iterations = True
                text += characters(b[0], self._vocabulary)[0]
                i_list.append(np.mean(self.num_iter.eval({self._sample_input: b[0]})))
                letters_parsed += 1
                if letters_parsed >= length:
                    collect_iterations = False
                    iterations_list.append(i_list)
                    text_list.append(text)
                    letters_parsed = -1
                    
            _ = self._sample_prediction.eval({self._sample_input: b[0]})
        return text_list, iterations_list  
    

In [22]:
model = ACT_LSTM_individual_2(64,
            vocabulary,
            characters_positions_in_vocabulary,
            20,
            [128],
                    5,
                    0.01,
                    0.00003,
            train_text,
            {'html': valid_text_1, 'text': valid_text_2},
            init_learning_rate=5.)

In [19]:
print_steps = [1000 * i for i in range(200)]
model.run(10,
          0.9,
            1000,
            50,
            3,
            1,
            100,
          #add_operations = ['self.number_of_iterations'],
          #add_text_operations=['self.first_inputs', 'self.first_outputs'],
          #print_steps = print_steps,
          #validation_add_operations=['self.probs'],
           #validation_add_operations=['self.num_iter', 'self.probs', 'self.halting_units'],
          #validation_add_operations=['self.num_iter'],
          #num_validation_prints=10,
            print_intermediate_results = True,
          validation_example_length=10)

Initialized
Average loss at step 0: 5.277689 learning rate: 0.500000
Percentage_of correct: 0.00%

random:
áfÆ¡JÂXnÜ«ùHjÑb^ì3@ëÞ#sx'^MµbuoÏ3õmb+~õùïÐ÷ClCÆwÕ]"!$yÂaaÄIu£cø>>uTy¢[i;óËï`#øoº
Gé9ýy=%tÈ¶çúÓkRZ@t9púÉrBÇWM°oü­ª½â-^´|ÁÙ¿ké«+.Íd&­Ã=aEÏùlJT&ÀÒafóõraIÄ'ÍØÈïZiå¥
p	?LCÈc¸]éÝaÊ°¸é;nþ&Ò¶=Ü	É¸	ÕÈO¥"æúÛAÂ?uâÐ¥L
·AV×ê620ù.0É¤@;¸qü¦h±hí(¢ÒH& Ó)$mó
.Õ×~ïSúç0$B]¡<ì¢úäl^«	"ô|	8áßå#mÓFá±Å+­g¼ Ãñ®¢Ñâ"uH«É=«òß|-/.=&Ôfx_äò¤DÍ[ï¾aõðef
$¼îªÆV\ºÅÎvG)¾¸uõanËU´Bvm¬DÖ¤µR"ùH÷éCÈRÉPVù¸Ìæõx°÷èc	á?«	cl-Oió
Ð_Èe²òÝY[ñ;Ü


KeyboardInterrupt: 

In [23]:
model.simple_run(200,
                   'ACT_LSTM_individual_2.0/variables/first_try_idt0.01_mf0.00003',
                   20000,
                   4000,
                   5000,        #learning has a chance to be stopped after every block of steps
                   40,
                   0.9,
                   3,
                   fixed_num_steps=True)

Number of steps = 20000     Percentage = 52.68%     Time = 10889s     Learning rate = 0.0739
